In [1]:
import os
import sys
# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../src"))

In [2]:
# Import functions from the model_training script
from model_training import preprocess_target_column, split_data, train_logistic_regression, train_random_forest, evaluate_model
import pandas as pd

In [3]:
#Load the processed data
processed_data_path = "D:/Kifya_training/Week 6/Bati-Bank-Credit-Scoring-Model/data/processed/processed_data.csv"
df = pd.read_csv(processed_data_path)

In [4]:
# Preprocess the target column
df = preprocess_target_column(df, "FraudResult")

In [5]:
print("Unique values in target column (FraudResult) after preprocessing:")
print(df["FraudResult"].unique())
print("Unique values in target column (FraudResult) after preprocessing:")
print(df["FraudResult"].unique())


Unique values in target column (FraudResult) after preprocessing:
[0 1]
Unique values in target column (FraudResult) after preprocessing:
[0 1]


In [6]:
# Define the target column and split the data
target_column = "FraudResult"
X_train, X_test, y_train, y_test = split_data(df, target_column)

In [7]:
# Train Logistic Regression
log_model = train_logistic_regression(X_train, y_train)
print("Logistic Regression Model Trained.")

Logistic Regression Model Trained.
